###MODEL EVAULATION AND KEYNOTES
In this notebook, we analyzed some models developed to understand different  approaches and techniques.

In [1]:
# install datasets
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [4]:
train_dataset[0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [5]:
label_dict = {
    '4': "positive",
    '3': "positive",
    '2': "neutral",
    '1': "negative",
    '0': "negative"
}
binary_label_dict = {
    "0" : "negative",
    "1" : "positive"
}

###UTILS

In [23]:
def get_predictions(A, tokenizer, model):

  truth = []
  predict = []

  for i, row in enumerate(test_dataset):
    if i>A:
      break
    text = row['text']
    label = row['label']
    truth.append(label_dict[str(label)])
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    # Forward pass through the model
    outputs = model(**inputs)

    # Get predicted probabilities
    probs = torch.softmax(outputs.logits, dim=1)

    # Get predicted label (index with highest probability)
    predicted_label = torch.argmax(probs, dim=1).item()
    predict.append(binary_label_dict[str(predicted_label)])

  return truth, predict

In [38]:
def get_predictions_ranking(A, tokenizer, model):

  truth = []
  predict = []

  for i, row in enumerate(test_dataset):
    if i>A:
      break
    text = row['text']
    label = row['label']
    truth.append(str(label))
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    # Forward pass through the model
    outputs = model(**inputs)

    # Get predicted probabilities
    probs = torch.softmax(outputs.logits, dim=1)

    # Get predicted label (index with highest probability)
    predicted_label = torch.argmax(probs, dim=1).item()
    predict.append(str(predicted_label))

  return truth, predict

In [24]:
def calculate_acc(truth,predict):
  total = 0
  count = 0
  for i, p in enumerate(predict):
    if truth[i] == "neutral":
      continue
    total = total + 1
    if truth[i] == "positive" and p == "positive":
      count = count + 1
    if truth[i] == "negative" and p == "negative":
      count = count + 1

  print(count, total, count/total)


In [40]:
def calculate_acc_ranking(truth,predict):
  total = 0
  count = 0
  for i, p in enumerate(predict):
    total = total + 1
    if truth[i] == p:
      count = count + 1

  print(count, total, count/total)


###karimbkh/BERT_fineTuned_Sentiment_Classification_Yelp
* Fine-tuned BERT (Bidirectional Encoder Representations from Transformers) for sequence classification.
* Pre-trained Model: BERT-base-uncased.
* Input: Cleaned and preprocessed restaurant reviews.
* Output: Binary classification (positive or negative sentiment).
* Tokenization: BERT tokenizer with a maximum sequence length of 240 tokens.
* Optimizer: AdamW with a learning rate of 3e-5.
* Learning Rate Scheduler: Linear scheduler with no warmup steps.
* Loss Function: CrossEntropyLoss.
* Batch Size: 16.
* Number of Epochs: 2.

#### Data Preprocessing
1. Loaded Yelp reviews dataset and business dataset.
2. Merged datasets on the "business_id" column.
3. Removed unnecessary columns and duplicates.
4. Translated star ratings into binary sentiment labels (positive or negative).
5. Upsampled the minority class (negative sentiment) to address imbalanced data.
6. Cleaned text data by removing non-letters, converting to lowercase, and tokenizing.

#### Model Training
1. Split the dataset into training (70%), validation (15%), and test (15%) sets.
2. Tokenized, padded, and truncated input sequences.
3. Created attention masks to differentiate real tokens from padding.
4. Fine-tuned BERT using the specified hyperparameters.
5. Tracked training and validation accuracy and loss for each epoch.

#### Model Performance
Validation Accuracy: ≈ 97.5% - 97.8%
Test Accuracy: ≈ 97.8%
F1 Score: ≈ 97.8% - 97.9%

In [25]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("karimbkh/BERT_fineTuned_Sentiment_Classification_Yelp")
model = BertForSequenceClassification.from_pretrained("karimbkh/BERT_fineTuned_Sentiment_Classification_Yelp")

This model is used for binary classification of the reviews.

In [26]:
truth, predictions = get_predictions(20, tokenizer, model)

In [27]:
calculate_acc(truth, predictions)

14 17 0.8235294117647058


###pig4431/YELP_roBERTa_5E

This model is a fine-tuned version of roberta-base on the yelp_review_full dataset. It achieves the following results on the evaluation set:

Loss: 0.0995
Accuracy: 0.9867

Training hyperparameters
The following hyperparameters were used during training:

* learning_rate: 1e-05
* train_batch_size: 16
* eval_batch_size: 8
* seed: 42
* optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
* lr_scheduler_type: linear
* num_epochs: 5

In [28]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("pig4431/YELP_roBERTa_5E")
model = AutoModelForSequenceClassification.from_pretrained("pig4431/YELP_roBERTa_5E")

In [29]:
truth, predictions = get_predictions(20, tokenizer, model)

In [30]:
calculate_acc(truth, predictions)

16 17 0.9411764705882353


###pig4431/YELP_BERT_5E
This model is a fine-tuned version of bert-base-cased on the yelp_review_full dataset. It achieves the following results on the evaluation set:

Loss: 0.1867
Accuracy: 0.9733

Training hyperparameters
The following hyperparameters were used during training:

* learning_rate: 2e-05
* train_batch_size: 16
* eval_batch_size: 16
* seed: 42
* optimizer: Adam with betas=(0.9,0.999) and epsilon=1e-08
* lr_scheduler_type: linear
* num_epochs: 5

In [31]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("pig4431/YELP_BERT_5E")
model = AutoModelForSequenceClassification.from_pretrained("pig4431/YELP_BERT_5E")

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [34]:
truth, predictions = get_predictions(50, tokenizer, model)

In [35]:
calculate_acc(truth, predictions)

40 43 0.9302325581395349


###rttl-ai/bert-base-uncased-yelp-reviews
Model Description: This model is a fine-tune checkpoint of [bert-base-uncased](https://huggingface.co/bert-base-uncased), fine-tuned on yelp reviews.
Resources for more information:
* The model was pre-trained with task-adaptive pre-training [TAPT](https://arxiv.org/pdf/2004.10964.pdf) with an increased masking rate, no corruption strategy, and using WWM, following [this paper](https://aclanthology.org/2023.eacl-main.217.pdf)
* pre-trained on yelp reviews
* fine-tuned on yelp reviews for multi-class text classification

ACC: 0.704

In [36]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("rttl-ai/bert-base-uncased-yelp-reviews")
model = AutoModelForSequenceClassification.from_pretrained("rttl-ai/bert-base-uncased-yelp-reviews")

tokenizer_config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [48]:
truth, predictions = get_predictions_ranking(50, tokenizer, model)

In [ ]:
print(predictions)
truth

In [50]:
calculate_acc_ranking(truth, predictions)

36 51 0.7058823529411765


###Other Models
Other models have different accuracy scores varies from 0.6 to 0.8. Lists of models that is worth to research:
* [Ramamurthi/distilbert-base-uncased-finetuned-yelp-reviews](https://huggingface.co/Ramamurthi/distilbert-base-uncased-finetuned-yelp-reviews): Star Prediction
* [Shunian/yelp_review_rating_reberta_base](https://huggingface.co/Shunian/yelp_review_rating_reberta_base): Star Prediction
* [DLochmelis33/22s-dl-sentiment-1](https://huggingface.co/DLochmelis33/22s-dl-sentiment-1?text=I+like+you.+I+love+you) : Binary Classification (Positive-Negative)

As you can see accuracy for star rating is much lower than the binary classification for sentiment analysis. In this perspective, we choose to work on star prediction because it would be more challenging and valuable at this point.

###SUGGESTIONS FOR TRAINING STEPS

#### Data Preprocessing:
* Tokenization: Convert each review text into a sequence of tokens (words or subwords).
* Padding: Ensure that all sequences are of equal length by padding shorter sequences with a special token (usually <pad>).
* Truncation: Optionally truncate longer sequences to a maximum length to fit the model's input size.

#### Model Selection:
* Transformer-based models: Models like BERT, RoBERTa, or DistilBERT have shown strong performance on various NLP tasks, including sentiment analysis.
* Sequence-to-sequence models: Models like LSTM or GRU can also be used for sequence prediction tasks, although they might not capture contextual information as effectively as transformers.

#### Fine-tuning Strategy:
* Pre-trained models: Start with a pre-trained language model (e.g., BERT) and fine-tune it on the Yelp dataset for star prediction.
* Learning rate: Experiment with different learning rates during fine-tuning to achieve optimal performance.
* Batch size: Adjust the batch size based on the available computational resources and model performance.

#### Loss Function:
* Cross-entropy loss: Commonly used for classification tasks like star prediction. It measures the difference between predicted and actual star ratings.

#### Evaluation Metrics:
* Accuracy: The proportion of correctly predicted star ratings.
* Mean Absolute Error (MAE): The average absolute difference between predicted and actual star ratings.
* Root Mean Squared Error (RMSE): The square root of the average squared difference between predicted and actual star ratings.

####Handling Class Imbalance:
* Check for class imbalance in the dataset, as there may be more reviews for certain star ratings than others.
* Techniques such as class weighting or oversampling/undersampling can be used to address class imbalances.

#### Model Interpretability:
* Analyze attention weights: Transformer models offer interpretability through attention mechanisms, which can help identify important words or phrases in reviews that contribute to star predictions.
* Feature importance: For traditional machine learning models, consider analyzing feature importance to understand which features are most influential in star prediction.

#### Hyperparameter Tuning:
* Experiment with different hyperparameters such as learning rate, batch size, dropout rate, and number of layers to optimize model performance.
* Use techniques like grid search or random search to efficiently search the hyperparameter space.

#### Cross-Validation:
* Use techniques like k-fold cross-validation to assess model generalization and robustness.
* Split the dataset into training, validation, and test sets to prevent overfitting and evaluate the model's performance on unseen data.

#### Post-processing:
* Threshold adjustment: For binary classification tasks (e.g., positive/negative sentiment), adjust the threshold for converting predicted probabilities into class labels to optimize performance.
* Error analysis: Analyze misclassified instances to identify patterns or biases in the model's predictions and refine the model accordingly.